<a href="https://colab.research.google.com/github/beza-lab/KAIM-WEEK-5/blob/main/Task4_Model_Comparison_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load
import torch

In [8]:
from google.colab import files

# This will prompt you to upload the file
uploaded = files.upload()

Saving labeled_dataset2.conll to labeled_dataset2.conll


In [ ]:
import pandas as pd

# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
print(f'Loading file: {filename}')

# Initialize lists to hold tokens and labels
tokens = []
labels = []

# Read the file line by line
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(maxsplit=1)  # Split by the first space only
            if len(parts) == 2:  # Ensure there are exactly 2 parts
                token, label = parts
                tokens.append(token)
                labels.append(label)
            else:
                print(f"Skipping line: {line}")  # Debugging output for lines that don't match the format

# Create a DataFrame
df = pd.DataFrame({'Token': tokens, 'Label': labels})





In [11]:
# Create a label mapping
label_mapping = {
    "O": 0,
    "I-LOC": 1,
    "I-PRICE": 2,
    "I-Product": 3,
    # Add other labels as needed
}

# Map the labels in the DataFrame
df['Label'] = df['Label'].map(label_mapping)

# Drop any rows with NaN values after mapping
df.dropna(inplace=True)

# Convert labels to integer type
df['Label'] = df['Label'].astype(int)


In [ ]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load
import torch

# Load the fast tokenizer and model for XLM-Roberta
xlm_roberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
xlm_roberta_model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_mapping))

def tokenize_and_align_labels_xlm_roberta(examples):
    tokenized_inputs = xlm_roberta_tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(examples['Label'].values[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_datasets_xlm_roberta = [tokenize_and_align_labels_xlm_roberta(sentence) for _, sentence in df.groupby(df.index // 10)]

# Convert lists to PyTorch tensors
input_ids_xlm_roberta = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]
attention_mask_xlm_roberta = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]
labels_xlm_roberta = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]

# Pad sequences to ensure they have the same length
input_ids_xlm_roberta = torch.nn.utils.rnn.pad_sequence(input_ids_xlm_roberta, batch_first=True, padding_value=xlm_roberta_tokenizer.pad_token_id)
attention_mask_xlm_roberta = torch.nn.utils.rnn.pad_sequence(attention_mask_xlm_roberta, batch_first=True, padding_value=0)
labels_xlm_roberta = torch.nn.utils.rnn.pad_sequence(labels_xlm_roberta, batch_first=True, padding_value=-100)

# Combine into a dataset
dataset_xlm_roberta = Dataset.from_dict({
    'input_ids': input_ids_xlm_roberta,
    'attention_mask': attention_mask_xlm_roberta,
    'labels': labels_xlm_roberta
})

# Set up training arguments for XLM-Roberta
training_args_xlm_roberta = TrainingArguments(
    output_dir='./results_xlm_roberta',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer_xlm_roberta = Trainer(
    model=xlm_roberta_model,
    args=training_args_xlm_roberta,
    train_dataset=dataset_xlm_roberta,
    eval_dataset=dataset_xlm_roberta,
)

trainer_xlm_roberta.train()

# Evaluate the model
results_xlm_roberta = trainer_xlm_roberta.evaluate()
print(results_xlm_roberta)

# Save the model
xlm_roberta_model.save_pretrained('./fine-tuned-xlm-roberta-model')
xlm_roberta_tokenizer.save_pretrained('./fine-tuned-xlm-roberta-model')


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
